In [14]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

import torch
from pydantic import BaseModel, ValidationError
from pydantic import field_validator
from typing import Any

- *Vamos a crear una modelo con Pydantic para validar el tipo de dato y tamaño de nuestros tensores.*
- *Un modelo es una clase que nos permite definir la estructura de nuestros datos y las reglas que vamos a utilizar para validarlos. Se definen como una sub-clase de la clase `BaseModel`.*

In [15]:
class TensorModel(BaseModel):
    tensor: Any

    @field_validator('tensor')
    @classmethod
    def validate_tensor(cls, value):
        if not isinstance(value, torch.Tensor):
            raise ValueError("Value must be a torch.Tensor object.")
        if value.dtype != torch.float32:
            raise ValueError("Tensor must be of type torch.float32.")
        if value.dim() != 2:
            raise ValueError("Tensor must be a 2D tensor.")
        if value.size(0) == 0 or value.size(1) == 0:
            raise ValueError("Tensor must not be empty.")
        return value

- *Pydantic no reconoce el tipo de dato `torch.Tensor`, por eso lo definimos con el tipo de dato `Any`. De esta forma le estamos diciendo a Pydantic que la variable `tensor` puede ser "de cualquier tipo". La consecuencia es que la validación la tenemos que definir nosotros utilizando `field_validator()`.*

In [16]:
# This will raise a ValueError if the tensor does not meet the criteria
try:
    tensor = torch.ones((3, 4), dtype=torch.float32)
    tensor = TensorModel(tensor=tensor)
    logging.info("Tensor is valid")
except ValidationError as error:
    logging.info("Validation error:", error)

2025-06-19 23:58:33,917 - INFO - Tensor is valid


In [19]:
# This will raise an error due to incorrect dimensions
try:
    tensor = torch.ones((2, 3, 4), dtype=torch.float32)
    tensor = TensorModel(tensor=tensor)
    logging.info("Tensor is valid")
except ValidationError as error:
    print("Validation error:", error)

Validation error: 1 validation error for TensorModel
tensor
  Value error, Tensor must be a 2D tensor. [type=value_error, input_value=tensor([[[1., 1., 1., 1.]...     [1., 1., 1., 1.]]]), input_type=Tensor]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
